# Lab 4: TableGPT

In this lab, we'll discover the power of code generation models through TableGPT2. The aim is to see how the model can be used in data analysis.

First of all, the notebook is divided into X sections: 0. Installation: This section is dedicated to module installation, model loading and data loading.

1. Guided introduction: Together, we'll discover how to use and evaluate TableGPT2.
2. More questions: You'll need to add at least one new question type to our simple evaluation system.
3. More data sets: You'll need to implement a question with multiple datasets.

IMPORTANT:

- You must work in pairs. You must submit **ONLY ONE NOTEBOOK** for each pair.
- Do not share your work with other pairs.
- You should not use Copilot, ChatGPT or similar tools. At the very least, remove the prompt ...
- <font color='red'>All the things you need to do are indicated in red.</font>

<font color='red'>**FIRST QUESTION:** What are the specificty of the TableGPT2 model?</font> https://huggingface.co/tablegpt/TableGPT2-7B


## 0. Setup


In [22]:
# !pip install transformers datasets bitsandbytes accelerate

In [23]:
from transformers import (
    BitsAndBytesConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    GenerationConfig,
)

import pandas as pd
import torch

In [25]:
llm_name = "tablegpt/TableGPT2-7B"

# We want to use 4bit quantization to save memory
quantization_config = BitsAndBytesConfig(load_in_8bit=False, load_in_4bit=True)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(llm_name, padding_side="left")
# Prevent some transformers specific issues.
tokenizer.use_default_system_prompt = False
tokenizer.pad_token_id = tokenizer.eos_token_id

# Load LLM.
llm = AutoModelForCausalLM.from_pretrained(
    llm_name,
    quantization_config=quantization_config,
    device_map="cuda",  # load all the model layers on GPU 0
    torch_dtype=torch.bfloat16,  # float precision
)
# Set LLM on eval mode.
llm.eval()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.02 GiB. GPU 0 has a total capacity of 15.89 GiB of which 26.12 MiB is free. Including non-PyTorch memory, this process has 15.87 GiB memory in use. Of the allocated memory 15.23 GiB is allocated by PyTorch, and 361.91 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [4]:
generation_config = GenerationConfig(
    max_new_tokens=512,
    do_sample=False,
    # do_sample=True,
    # temperature=.7,
    # top_p=.8,
    # top_k=20,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)

In [5]:
df = pd.read_csv("hf://datasets/phihung/titanic/train.csv")
df = df.drop("Cabin", axis=1).dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  712 non-null    int64  
 1   Survived     712 non-null    int64  
 2   Pclass       712 non-null    int64  
 3   Name         712 non-null    object 
 4   Sex          712 non-null    object 
 5   Age          712 non-null    float64
 6   SibSp        712 non-null    int64  
 7   Parch        712 non-null    int64  
 8   Ticket       712 non-null    object 
 9   Fare         712 non-null    float64
 10  Embarked     712 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 66.8+ KB


## 1.1 Guided Introduction: The Model.

Below there is an example of a prompt that could be used with TableGPT2.

```
Given access to several pandas dataframes, write the Python code to answer the user's question.
The answer should be store in a variable named "output".

/*
"df.head(5).to_string(index=False)" as follows:
 PassengerId  Survived  Pclass                                                Name    Sex  Age  SibSp  Parch           Ticket    Fare Embarked
           1         0       3                             Braund, Mr. Owen Harris   male 22.0      1      0        A/5 21171  7.2500        S
           2         1       1 Cumings, Mrs. John Bradley (Florence Briggs Thayer) female 38.0      1      0         PC 17599 71.2833        C
           3         1       3                              Heikkinen, Miss. Laina female 26.0      0      0 STON/O2. 3101282  7.9250        S
           4         1       1        Futrelle, Mrs. Jacques Heath (Lily May Peel) female 35.0      1      0           113803 53.1000        S
           5         0       3                            Allen, Mr. William Henry   male 35.0      0      0           373450  8.0500        S
*/

Question: How many child survive? (under 18)
```

The prompt is divided in 3 parts:

1. The global instruction wich is to write python that could answer a question on a specific dataset.
2. The header of the given dataset: 5 first lines of titanic dataset.
3. The question to answer: "How many child survive? (under 18)

First, we will implement a function that generate an answer for this prompt.

<font color='red'>TODO: Fill in the `generate_answer` function following the comments inside.</font>


In [21]:
example_prompt_template = """Given access to several pandas dataframes, write the Python code to answer the user's question.
The answer should be store in a variable named "output".

/*
"{var_name}.head(5).to_string(index=False)" as follows:
{df_info}
*/

Question: {user_question}
"""


def generate_answer(prompt, llm=llm, generation_config=generation_config):
    # Create turns with the given prompt.
    chat = [
        {"role": "user", "content": prompt},
    ]

    # Apply template with the tokenizer. Be careful to return pt tensors on the same device than `llm`.
    chat_encoded = tokenizer.apply_chat_template(
        chat, return_tensors="pt", padding="max_length"
    ).to(llm.device)

    # Generate with llm using the given generation config.
    llm_outputs = llm.generate(
        input_ids=chat_encoded,
        generation_config=generation_config,
    )

    # Decode and select the answer to return.
    answer = tokenizer.decode(llm_outputs[0], skip_special_tokens=True)
    print(answer)
    return answer


prompt = example_prompt_template.format(
    var_name="df",
    df_info=df.head(5).to_string(index=False),
    user_question="How many child survive? (under 18)",
)

answer = generate_answer(prompt)

print(prompt)
print("\n*****\n")
print(answer)

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.75 GiB. GPU 0 has a total capacity of 15.89 GiB of which 1.07 GiB is free. Including non-PyTorch memory, this process has 14.82 GiB memory in use. Of the allocated memory 12.78 GiB is allocated by PyTorch, and 1.76 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## 1.2 Guided Introduction: The Answer.

As you can see, the model answer with some generated code.

````
Python code:
```python
# Filter the dataframe to include only passengers under the age of 18
children = df[df['Age'] < 18]

# Count the number of children who survived
child_survivors = children[children['Survived'] == 1]

# Save the answer in the variable output
output = len(child_survivors)
````

So we will need to execute it, but there is some difficulty:

1. Sometime, the llm answer with \`\`\`python ... \`\`\`, sometime the llm answer directly with the code. We need to handle both cases.
2. We need to recover the variable output from the execution.
3. We need to evaluate single value and list of values.

First, we will implement a function that generate an answer for this prompt.

<font color='red'>TODO: Fill in the `exec_answer` function following the comments inside.</font>


In [ ]:
import re

def exec_answer(answer, gold):

  # Extract the code from the answer. Be careful, the code is now always in ``` ```.

  # Execute the code, https://docs.python.org/3/library/functions.html#exec
  # if the code work: Return True or False based on output == gold (be careful to handle iterable !)
  # if the code don't work return False.


print(exec_answer(answer, 61))

## 1.3 Guided Introduction: The Question.

Now we want to automatically generate questions to evaluate the performance of our model. There are benchmarks on this subject, but here we want to practice code by generating the questions ourselves.

We will generate some basic filter questions.

<font color='red'>TODO: Fill in the `generate_filter_question` function following the comments inside.</font>


In [ ]:
import random

def generate_random_question(generate_function, df, k=1, seed=42):
  random.seed(seed)
  return [generate_function(df) for _ in range(k)]

def generate_filter_question(df):

  # Create a question template that take a target colunm, a filter column and a filter value

  # Get a random target column and a random filter column (be careful they should be differnts)

  # Get a random filter value inside the filer column. Avoid NaN values.

  # Compute the correct answer for the given target column, filter column and filter value.

  # return formated question and associated answer in a dict {"question":[question], "answer":[answer]}

generate_random_question(generate_filter_question, df, k=5)

## 1.4 Guided Introduction: The Evaluation.

The last step in this section is to evaluate our model on 20 random questions! We'll use simple accuracy.

You should have an accuracy between 0.9 and 1.

<font color='red'>TODO: Follow instruction in comment of the cell below.</font>

<font color='green'>BONUS: Investigate on errors and improve our prompt/parsing to solve them.</font>


In [ ]:
from tqdm import tqdm

# Generate 20 random question

# Iterate over question to format prompt, generate answer and execute answer.
# Report the Accuracy

print(
    "Acc: ",
)

## 2. More Questions.

Now it's your turn to imagine a type of question ("How many ..."). Implement a function to generate new type of question. Verify that our previous code work with your new question then evaluate it.

<font color='red'>TODO: Generate **AT LEAST ONE** new type of question and report this new question accuracy.</font>


## 3. More datasets.

Below we load a new dataset: "adult_income_dataset".

<font color='red'>TODO: Evaluate our questions on this new dataset. Report the accuracy. Comment Any differences.</font>

<font color='green'>BONUS: Try to find a prompt that answer this question: What is the mean salary of titanic surviror based on adult dataset.</font>


In [ ]:
adult = pd.read_csv("hf://datasets/meghana/adult_income_dataset/adult.csv")
adult.info()

titanic = df